## Header

### Import Library

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib
from binance.client import Client
from datetime import datetime

### Define Wallet Class

In [15]:
class Wallet:
    def __init__(self, initial_balance):
        self.balance = initial_balance
        self.coins = {}

    def add_coin(self, name, quantity, value):
        cost = quantity * value
        # Check if have enough money
        if self.balance > cost:
            self.balance -= cost
            if name in self.coins:
                self.coins[name] += quantity
            else:
                self.coins[name] = quantity
            return True
        else:
            return False

    def remove_coin(self, name, quantity, value):
        if name in self.coins:
            if self.coins[name] >= quantity:
                cost = quantity * value
                self.balance += cost
                self.coins[name] -= quantity
                if self.coins[name] == 0:
                    del self.coins[name]
                return True
            else:
                return False
        else:
            return False

    def get_coin_quantity(self, name):
        if name in self.coins:
            return self.coins[name]
        else:
            return 0

    def get_balance(self):
        return self.balance
    
    def get_coins(self):
        return self.coins

### Define Strategy

In [16]:
class Strategy:
    # Momentum Trading Strategy

    def __init__(self, short_window, long_window, rsi_window):
        self.short_window = short_window
        self.long_window = long_window
        self.rsi_window = rsi_window
        self.prices = pd.DataFrame(columns=['price',
                                            'short_mavg',
                                            'long_mavg',
                                            'rsi'])
        self.price_history = []

    def update_price_history(self, price):
        self.price_history.append(price)
        if len(self.price_history) == self.long_window:
            self.price_history.pop(0)
            return True
        return False

    def generate_signals(self, price):
        new_row = {'price': price, 'short_mavg': None, 'long_mavg' : None, 'rsi': None}

        if self.update_price_history(price):
            short_mavg = np.mean(self.price_history[-self.short_window:])
            long_mavg  = np.mean(self.price_history)
            rsi = talib.RSI(np.array(self.price_history), self.rsi_window)[-1]

            new_row = {'price': price,
                       'short_mavg': short_mavg,
                       'long_mavg' : long_mavg,
                       'rsi': rsi}
            
            self.prices.loc[len(self.prices)] = new_row
            if short_mavg > long_mavg and rsi > 70:
                return 'BUY'
            if short_mavg < long_mavg and rsi < 30:
                return 'SELL'
            
            
        self.prices.loc[len(self.prices)] = new_row

    def get_prices(self):
        return self.prices

### Define other classes and functions

#### Fetch Data

In [17]:
def fetchData(client, symbol, interval, start_time, end_time):
    klines = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start_time, end_str=end_time)
    close_prices = [float(kline[4]) for kline in klines]
    timestamps = [datetime.fromtimestamp(kline[0] / 1000) for kline in klines]

    data = pd.DataFrame({'Timestamp': timestamps, 'Close Price': close_prices})

    return data

#### Simulating

In [18]:
def simulation(data, wallet, strategy, symbol):
    data_balance = [None] * len(data)
    data_signal = [None] * len(data)

    coin_name = symbol.replace('USDT', '')

    for row in data.itertuples():
        data_index = row[0]
        # row[1] is timestamp
        close_price = row[2]

        signal = strategy.generate_signals(close_price)

        if signal == 'BUY':
            available_balance = wallet.get_balance()
            if available_balance > 1:
                quantity = available_balance / close_price
                if wallet.add_coin(coin_name, quantity, close_price):
                    data_signal[data_index] = 'BUY'

        elif signal == 'SELL':
            coin_pair = wallet.get_coins()
            if len(coin_pair) > 0:
                for coin_name_in_wallet, coin_qty_in_wallet in coin_pair.items():
                    if coin_name == coin_name_in_wallet:
                        if wallet.remove_coin(coin_name, coin_qty_in_wallet, close_price):
                            data_signal[data_index] = 'SELL'
                        break
        
        data_balance[data_index] = wallet.get_balance()

    # Force sell all coin
    coin_pair = wallet.get_coins()
    if len(coin_pair) > 0:
        for coin_name_in_wallet, coin_qty_in_wallet in coin_pair.items():
            if coin_name == coin_name_in_wallet:
                if wallet.remove_coin(coin_name, coin_qty_in_wallet, close_price):
                    data_signal[data_index] = 'SELL'
                    data_balance[-1] = wallet.get_balance()
                break

    data['Signal'] = data_signal
    data['Balance'] = data_balance
    return data

## Backtesting Strategy

### Parameters

#### Input Parameters

In [19]:
# interval = Client.KLINE_INTERVAL_1HOUR
interval = Client.KLINE_INTERVAL_1DAY
start_time_train = int(datetime(2020,1,1,0,0).timestamp() * 1000)
start_time_train_plus1 = int(datetime(2020,1,2,0,0).timestamp() * 1000)
end_time_train = int(datetime(2021,12,31,0,0).timestamp() * 1000)

start_time_val = int(datetime(2022,1,1,0,0).timestamp() * 1000)
end_time_val_minus1 = int(datetime(2023,12,1,0,0).timestamp() * 1000)
end_time_val = int(datetime(2023,12,31,0,0).timestamp() * 1000)

initial_balance = 1000

### Initialise API

In [20]:
client = Client(API_KEY, API_SECRET)

#### Fetch Training data set from binance

In [21]:
# ticker_infos = client.get_ticker()
# symbols = [symbol['symbol'] for symbol in ticker_infos if symbol['symbol'].endswith('USDT')]

In [22]:
# symbols_remove = []
# for symbol in symbols:
#     data_test = fetchData(client, symbol, Client.KLINE_INTERVAL_1DAY, start_time_train, start_time_train_plus1)
#     if data_test.empty:
#         symbols_remove.append(symbol)
#         continue
#     data_test = fetchData(client, symbol, interval, end_time_val_minus1, end_time_val)
#     if data_test.empty:
#         symbols_remove.append(symbol)

# for symbol in symbols_remove:
#     symbols.remove(symbol)

## Results

### Optimizing

In [27]:
optimize_results = pd.DataFrame(columns=['symbol',
                                         'profit margin',
                                         'short window',
                                         'long window',
                                         'rsi window'])

symbols = ['BTCUSDT']

for symbol in symbols:
    data_train = fetchData(client, symbol, interval, start_time_train, end_time_train)
    data_val   = fetchData(client, symbol, interval, start_time_val  , end_time_val)
    
    # Select hyperparameter

    max_profit = 0
    short_selected = 0
    long_selected  = 0
    rsi_selected   = 0

    for window_size in range(10, 41, 5):
        for i in range(16, 31, 2):
            for j in range(5, 16):
                wallet         = Wallet(initial_balance=initial_balance)

                short_window     = window_size
                short_long_ratio = i / 10
                rsi_ratio        = j / 10
                long_window      = int(window_size * short_long_ratio)
                rsi_window       = int(window_size * rsi_ratio)
                strategy         = Strategy(short_window=short_window, long_window=long_window, rsi_window=rsi_window)

                sim            = simulation(data_train, wallet, strategy, symbol)
                ending_balance = sim['Balance'].iloc[-1]
                if ending_balance > max_profit:
                    short_selected = short_window
                    long_selected  = long_window
                    rsi_selected   = rsi_window

                    max_profit = ending_balance
            
    # Evaluate profitability

    wallet = Wallet(initial_balance=initial_balance)

    short_window = short_selected
    long_window  = long_selected
    rsi_window   = rsi_selected
    strategy       = Strategy(short_window=short_window, long_window=long_window, rsi_window=rsi_window)
        
    sim = simulation(data_val, wallet, strategy, symbol)
    ending_balance = sim['Balance'].iloc[-1]
    profit_margin = ending_balance / initial_balance

    new_row = {'symbol': symbol,
               'profit margin': profit_margin,
               'short window' : short_window,
               'long window'  : long_window,
               'rsi window'   : rsi_window}

    optimize_results.loc[len(optimize_results)] = new_row

/var/folders/ft/h3fjv72s4jdftdb2szbgyggr0000gp/T/ipykernel_10324/3991230080.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.prices.loc[len(self.prices)] = new_row
/var/folders/ft/h3fjv72s4jdftdb2szbgyggr0000gp/T/ipykernel_10324/3991230080.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.prices.loc[len(self.prices)] = new_row
/var/folders/ft/h3fjv72s4jdftdb2szbgyggr0000gp/T/ipykernel_10324/3991230080.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or al

In [28]:
optimize_results

,symbol,profit margin,short window,long window,rsi window
0,BTCUSDT,0.902362,15,45,9


### Check

In [ ]:
symbol = 'BTCUSDT'
data_train = fetchData(client, symbol, interval, start_time_train, end_time_train)
data_val   = fetchData(client, symbol, interval, start_time_val  , end_time_val)

In [25]:
wallet = Wallet(initial_balance=1000)

short_window = 10
long_window  = 16
rsi_window   = 5
strategy       = Strategy(short_window=short_window, long_window=long_window, rsi_window=rsi_window)
    
sim = simulation(data_val, wallet, strategy, symbol)
ending_balance = sim['Balance'].iloc[-1]

ending_balance

/var/folders/ft/h3fjv72s4jdftdb2szbgyggr0000gp/T/ipykernel_10324/3991230080.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.prices.loc[len(self.prices)] = new_row


908.8114225921568